In [ ]:
# upload top-3 models
# make predictions
# re-use get_weather for next 12 hours function(should be added into utils package ) - half-half
# re-use get_isw_from_yesterday - not done yet at all I would say